# Baseline models on Meta data.

In [73]:
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import column_or_1d
from typing import Tuple, Any, Union
import random
import pickle
import uuid

In [74]:
##TODO: change directory to cwd if run on colab
%cd /content/drive/MyDrive/Colab Notebooks/vogel
%ls

/content/drive/MyDrive/Colab Notebooks/vogel
 Baseline.ipynb
'Copy of Glycomics_UGA4_Day0Day28.xlsx'
'Copy of Metabolomics_UGA4_D0toDay28.xlsx'
'Copy of metabolomics_UGA4_global_median_normalized_sample_only.quantified.txt'
'Copy of metabolomics_UGA4_untargeted_median_normalized_sample_only.quantified.txt'
'Copy of Proteomics_UGA4_Day0.xlsx'
'Copy of Transcriptomics_UGA4_day0_logCPMcounts.xlsx'
 data.arff
 data_inspect.ipynb
'fluvacc metadata - UGA1-5 - 1368 entries with clear vacc status - for figures.txt'
 meta.txt


In [75]:
df = pd.read_csv("fluvacc metadata - UGA1-5 - 1368 entries with clear vacc status - for figures.txt", sep="\t")  ##TODO: put the name of the file here
df.head()

,Cohort,ID,Age,BMI,BMI_category,Gender,Race,Comorbidities,PreVacc_status,PreVacc_status_year,...,SC_category_IBV_Yam,SC_category_IBV_Vic,Composite_SC_category,Serostatus_H1N1,Serostatus_H3N2,Serostatus_IBV_Yam,Serostatus_IBV_Vic,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline
0,UGA1,A10,25,33.71,Obese,Male,White,No,Prevaccinated,Year_0,...,None,None,None,1,1,1,1,4,High,35.0
1,UGA1,A100,47,33.37,Obese,Female,White,No,Prevaccinated,Year_0,...,None,High,Low,0,0,1,0,1,Low,18.0
2,UGA1,A101,30,24.99,Normal,Male,Non-white,Yes,Naive,Year_3,...,High,High,High,0,0,1,1,2,Low,21.0
3,UGA1,A102,47,25.00,Overweight,Male,White,No,Naive,Year_3,...,Low,High,High,1,1,1,0,3,High,25.0
4,UGA1,A103,20,22.10,Normal,Female,Non-white,No,Naive,Year_3,...,High,High,High,1,1,1,1,4,High,21.0


In [76]:
for column in df:
  print(column, sorted(df[column].unique()))

Cohort ['UGA1', 'UGA2', 'UGA3', 'UGA4', 'UGA5']
ID ['A1', 'A10', 'A100', 'A101', 'A102', 'A103', 'A104', 'A105', 'A106', 'A107', 'A108', 'A109', 'A11', 'A110', 'A111', 'A112', 'A113', 'A114', 'A115', 'A116', 'A118', 'A119', 'A120', 'A122', 'A123', 'A124', 'A125', 'A126', 'A127', 'A128', 'A129', 'A13', 'A130', 'A131', 'A132', 'A133', 'A134', 'A135', 'A136', 'A137', 'A138', 'A139', 'A14', 'A140', 'A141', 'A142', 'A143', 'A144', 'A145', 'A146', 'A147', 'A148', 'A149', 'A15', 'A150', 'A151', 'A152', 'A154', 'A155', 'A156', 'A157', 'A158', 'A16', 'A160', 'A161', 'A162', 'A163', 'A165', 'A166', 'A168', 'A169', 'A17', 'A170', 'A171', 'A172', 'A174', 'A175', 'A176', 'A177', 'A178', 'A179', 'A18', 'A180', 'A181', 'A182', 'A184', 'A185', 'A186', 'A188', 'A19', 'A190', 'A191', 'A192', 'A193', 'A194', 'A195', 'A196', 'A197', 'A199', 'A2', 'A20', 'A200', 'A201', 'A203', 'A204', 'A205', 'A206', 'A207', 'A208', 'A209', 'A21', 'A210', 'A211', 'A212', 'A213', 'A214', 'A215', 'A216', 'A217', 'A218', 'A2

# Basic Preprocessing

* Some entries for $\texttt{BMI_category}$ have unnecessary spaces, to fix:

In [77]:
df.loc[df["BMI_category"] == "Normal ", "BMI_category"] = "Normal"
df.loc[df["BMI_category"] == "Obese ", "BMI_category"] = "Obese"
for column in df:
  print(column, sorted(df[column].unique()))

Cohort ['UGA1', 'UGA2', 'UGA3', 'UGA4', 'UGA5']
ID ['A1', 'A10', 'A100', 'A101', 'A102', 'A103', 'A104', 'A105', 'A106', 'A107', 'A108', 'A109', 'A11', 'A110', 'A111', 'A112', 'A113', 'A114', 'A115', 'A116', 'A118', 'A119', 'A120', 'A122', 'A123', 'A124', 'A125', 'A126', 'A127', 'A128', 'A129', 'A13', 'A130', 'A131', 'A132', 'A133', 'A134', 'A135', 'A136', 'A137', 'A138', 'A139', 'A14', 'A140', 'A141', 'A142', 'A143', 'A144', 'A145', 'A146', 'A147', 'A148', 'A149', 'A15', 'A150', 'A151', 'A152', 'A154', 'A155', 'A156', 'A157', 'A158', 'A16', 'A160', 'A161', 'A162', 'A163', 'A165', 'A166', 'A168', 'A169', 'A17', 'A170', 'A171', 'A172', 'A174', 'A175', 'A176', 'A177', 'A178', 'A179', 'A18', 'A180', 'A181', 'A182', 'A184', 'A185', 'A186', 'A188', 'A19', 'A190', 'A191', 'A192', 'A193', 'A194', 'A195', 'A196', 'A197', 'A199', 'A2', 'A20', 'A200', 'A201', 'A203', 'A204', 'A205', 'A206', 'A207', 'A208', 'A209', 'A21', 'A210', 'A211', 'A212', 'A213', 'A214', 'A215', 'A216', 'A217', 'A218', 'A2

* Co-linear Columns
  Some columns are obviously correlated. The following shows perfect colinearity between $\texttt{PreVacc_status}$ and $\texttt{PreVacc_status_year}$:

In [78]:
print(df[df["PreVacc_status"] == "Naive"][["PreVacc_status_year"]].describe())
print(df[df["PreVacc_status"] == "Prevaccinated"][["PreVacc_status_year"]].describe())


       PreVacc_status_year
count                  200
unique                   1
top                 Year_3
freq                   200
       PreVacc_status_year
count                 1168
unique                   1
top                 Year_0
freq                  1168


This means that we can drop one of the two columns:

In [79]:
df = df.drop("PreVacc_status_year", axis=1)
df

,Cohort,ID,Age,BMI,BMI_category,Gender,Race,Comorbidities,PreVacc_status,Month_vaccinated,...,SC_category_IBV_Yam,SC_category_IBV_Vic,Composite_SC_category,Serostatus_H1N1,Serostatus_H3N2,Serostatus_IBV_Yam,Serostatus_IBV_Vic,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline
0,UGA1,A10,25,33.71,Obese,Male,White,No,Prevaccinated,Jan.,...,None,None,None,1,1,1,1,4,High,35.00
1,UGA1,A100,47,33.37,Obese,Female,White,No,Prevaccinated,Jan.,...,None,High,Low,0,0,1,0,1,Low,18.00
2,UGA1,A101,30,24.99,Normal,Male,Non-white,Yes,Naive,Jan.,...,High,High,High,0,0,1,1,2,Low,21.00
3,UGA1,A102,47,25.00,Overweight,Male,White,No,Naive,Feb.,...,Low,High,High,1,1,1,0,3,High,25.00
4,UGA1,A103,20,22.10,Normal,Female,Non-white,No,Naive,Jan.,...,High,High,High,1,1,1,1,4,High,21.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,UGA5,C90,14,19.77,Normal,Female,White,No,Prevaccinated,Oct.,...,Low,None,None,1,0,1,0,2,Low,20.29
1364,UGA5,C93,14,18.45,Lean,Male,White,No,Prevaccinated,Oct.,...,High,Low,Low,0,1,1,0,2,Low,20.29
1365,UGA5,C94,15,25.71,Overweight,Male,White,No,Prevaccinated,Nov.,...,High,High,High,1,0,1,1,3,High,20.29
1366,UGA5,C97,17,23.29,Normal,Male,White,No,Prevaccinated,Oct.,...,Low,High,Low,1,1,1,1,4,High,26.29


* Normalization
>Only normalize possible input columns

In [80]:
def normalize(data):
    scaler = StandardScaler()
    scaler.fit(data)
    data = scaler.transform(data)
    return data

numerical_cols = ["Age", "BMI","D0_Titer_H1N1","D0_Titer_H3N2", "D0_Titer_IBV_Yam", "D0_Titer_IBV_Vic"]
df[numerical_cols] = normalize(df[numerical_cols])
df

,Cohort,ID,Age,BMI,BMI_category,Gender,Race,Comorbidities,PreVacc_status,Month_vaccinated,...,SC_category_IBV_Yam,SC_category_IBV_Vic,Composite_SC_category,Serostatus_H1N1,Serostatus_H3N2,Serostatus_IBV_Yam,Serostatus_IBV_Vic,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline
0,UGA1,A10,-0.622445,1.039092,Obese,Male,White,No,Prevaccinated,Jan.,...,None,None,None,1,1,1,1,4,High,35.00
1,UGA1,A100,0.429208,0.986051,Obese,Female,White,No,Prevaccinated,Jan.,...,None,High,Low,0,0,1,0,1,Low,18.00
2,UGA1,A101,-0.383433,-0.321247,Normal,Male,Non-white,Yes,Naive,Jan.,...,High,High,High,0,0,1,1,2,Low,21.00
3,UGA1,A102,0.429208,-0.319687,Overweight,Male,White,No,Naive,Feb.,...,Low,High,High,1,1,1,0,3,High,25.00
4,UGA1,A103,-0.861457,-0.772093,Normal,Female,Non-white,No,Naive,Jan.,...,High,High,High,1,1,1,1,4,High,21.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,UGA5,C90,-1.148271,-1.135578,Normal,Female,White,No,Prevaccinated,Oct.,...,Low,None,None,1,0,1,0,2,Low,20.29
1364,UGA5,C93,-1.148271,-1.341501,Lean,Male,White,No,Prevaccinated,Oct.,...,High,Low,Low,0,1,1,0,2,Low,20.29
1365,UGA5,C94,-1.100469,-0.208925,Overweight,Male,White,No,Prevaccinated,Nov.,...,High,High,High,1,0,1,1,3,High,20.29
1366,UGA5,C97,-1.004864,-0.586451,Normal,Male,White,No,Prevaccinated,Oct.,...,Low,High,Low,1,1,1,1,4,High,26.29


* Encoding method for $\texttt{Month_vaccinated}$:
>$\texttt{True}$: one-hot\
$\texttt{False}$: label encoding


In [81]:
month_one_hot = True  # change and rerun the notebook for this to take effect

In [82]:
''' Deprecated. Was used to generate binary encoding for the column
df.loc[df["Month_vaccinated"] == "Sep.", "Month_vaccinated"] = 0
df.loc[df["Month_vaccinated"] == "Oct.", "Month_vaccinated"] = 0
df.loc[df["Month_vaccinated"] == "Nov.", "Month_vaccinated"] = 0

df.loc[df["Month_vaccinated"] == "Dec.", "Month_vaccinated"] = 1
df.loc[df["Month_vaccinated"] == "Jan.", "Month_vaccinated"] = 1
df.loc[df["Month_vaccinated"] == "Feb.", "Month_vaccinated"] = 1
df = df.astype({'Month_vaccinated': 'int32'})
df
'''

' Deprecated. Was used to generate binary encoding for the column\ndf.loc[df["Month_vaccinated"] == "Sep.", "Month_vaccinated"] = 0\ndf.loc[df["Month_vaccinated"] == "Oct.", "Month_vaccinated"] = 0\ndf.loc[df["Month_vaccinated"] == "Nov.", "Month_vaccinated"] = 0\n\ndf.loc[df["Month_vaccinated"] == "Dec.", "Month_vaccinated"] = 1\ndf.loc[df["Month_vaccinated"] == "Jan.", "Month_vaccinated"] = 1\ndf.loc[df["Month_vaccinated"] == "Feb.", "Month_vaccinated"] = 1\ndf = df.astype({\'Month_vaccinated\': \'int32\'})\ndf\n'

* Conserve some categorical columns whose categories are quantatively related.Regular one hot encoding on the other columns whose categories are not quantatively related. 

In [83]:
bmi_cat = df["BMI_category"]
df2 = df.drop("BMI_category", axis=1)
comorbidities = df["Comorbidities"]
df2 = df2.drop("Comorbidities", axis=1)
dose = df["Vaccine_dose"]
df2 = df2.drop("Vaccine_dose", axis=1)

if not month_one_hot:
  month = df["Month_vaccinated"]
  df2 = df2.drop("Month_vaccinated", axis=1)

pre_vac = df["PreVacc_status"]
df2 = df2.drop("PreVacc_status", axis=1)
h1n1 = df["SC_category_H1N1"]
df2 = df2.drop("SC_category_H1N1", axis=1)
h3n2 = df["SC_category_H3N2"]
df2 = df2.drop("SC_category_H3N2", axis=1)
yam = df["SC_category_IBV_Yam"]
df2 = df2.drop("SC_category_IBV_Yam", axis=1)
vic = df["SC_category_IBV_Vic"]
df2 = df2.drop("SC_category_IBV_Vic", axis=1)
comp = df["Composite_SC_category"]
df2 = df2.drop("Composite_SC_category", axis=1)
base = df["Baseline_category_Num_SeroPos_strains"]
df2 = df2.drop("Baseline_category_Num_SeroPos_strains", axis=1)
df2


,Cohort,ID,Age,BMI,Gender,Race,Month_vaccinated,D0_Titer_H1N1,D28_Titer_H1N1,D0_Titer_H3N2,...,Seroconversion_H3N2,Seroconversion_IBV_Yam,Seroconversion_IBV_Vic,Composite_seroconversion,Serostatus_H1N1,Serostatus_H3N2,Serostatus_IBV_Yam,Serostatus_IBV_Vic,Num_SeroPos_strains,Composite_baseline
0,UGA1,A10,-0.622445,1.039092,Male,White,Jan.,0.637952,160,3.591594,...,0.0,0,0.0,0.0,1,1,1,1,4,35.00
1,UGA1,A100,0.429208,0.986051,Female,White,Jan.,-0.448919,40,-0.447529,...,1.0,0,4.0,6.0,0,0,1,0,1,18.00
2,UGA1,A101,-0.383433,-0.321247,Male,Non-white,Jan.,-0.448919,640,-0.447529,...,4.0,3,6.0,18.0,0,0,1,1,2,21.00
3,UGA1,A102,0.429208,-0.319687,Male,White,Feb.,0.016883,160,-0.317235,...,1.0,1,5.0,8.0,1,1,1,0,3,25.00
4,UGA1,A103,-0.861457,-0.772093,Female,Non-white,Jan.,-0.293651,640,-0.317235,...,3.0,2,3.0,12.0,1,1,1,1,4,21.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,UGA5,C90,-1.148271,-1.135578,Female,White,Oct.,0.016883,40,-0.447529,...,1.0,1,0.0,1.0,1,0,1,0,2,20.29
1364,UGA5,C93,-1.148271,-1.341501,Male,White,Oct.,-0.448919,160,-0.317235,...,1.0,2,1.0,7.0,0,1,1,0,2,20.29
1365,UGA5,C94,-1.100469,-0.208925,Male,White,Nov.,-0.293651,80,-0.512677,...,5.0,2,2.0,10.0,1,0,1,1,3,20.29
1366,UGA5,C97,-1.004864,-0.586451,Male,White,Oct.,0.016883,160,0.464531,...,1.0,1,2.0,5.0,1,1,1,1,4,26.29


In [84]:
df2.select_dtypes(exclude=["number"])

,Cohort,ID,Gender,Race,Month_vaccinated
0,UGA1,A10,Male,White,Jan.
1,UGA1,A100,Female,White,Jan.
2,UGA1,A101,Male,Non-white,Jan.
3,UGA1,A102,Male,White,Feb.
4,UGA1,A103,Female,Non-white,Jan.
...,...,...,...,...,...
1363,UGA5,C90,Female,White,Oct.
1364,UGA5,C93,Male,White,Oct.
1365,UGA5,C94,Male,White,Nov.
1366,UGA5,C97,Male,White,Oct.


* Now turn categorical columns into one hot encoding, dropping one role for each column to avoid perfect colinearity. For BMI category, I used label encoder.

In [85]:
for i, column in enumerate(df2.select_dtypes(exclude=["number"]).iloc[:,2:].columns):
  temp = pd.get_dummies(df2[column], drop_first=True, prefix=column)
  df = df.drop(column, axis=1)
  if i != 0 :
    x = x.join(temp)
  else:
    x = temp
x



,Gender_Male,Race_White,Month_vaccinated_Feb.,Month_vaccinated_Jan.,Month_vaccinated_Nov.,Month_vaccinated_Oct.,Month_vaccinated_Sep.
0,1,1,0,1,0,0,0
1,0,1,0,1,0,0,0
2,1,0,0,1,0,0,0
3,1,1,1,0,0,0,0
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
1363,0,1,0,0,0,1,0
1364,1,1,0,0,0,1,0
1365,1,1,0,0,1,0,0
1366,1,1,0,0,0,1,0


* Deal with the rest of categorical columns with label encoder
>*   For each column we want a specific ordering of the values indicating intensity
  *   Need to modify the label encoder



In [86]:
class CustomLabelEncoder(LabelEncoder):
  def fit(self, y, ordering: list):  # take an ordering list as an argument
      y = column_or_1d(y, warn=True)
      assert set(ordering) == set(pd.Series(y).unique())
      self.classes_ = ordering
      return self
if not month_one_hot:
  ordering_dict = {"BMI_category": (bmi_cat, ["Lean", "Normal", "Overweight", "Obese"]),
                 "Comorbidities": (comorbidities, ["No", "Yes"]),
                 "Vaccine_dose": (dose, ["Standard", "High"]),

                 "Month_vaccinated": (month, ["Sep.", "Oct.", "Nov.", "Dec.", "Jan.", "Feb."]),

                 "PreVacc_status": (pre_vac, ["Naive", "Prevaccinated"]),
                 "SC_category_H1N1": (h1n1, ["None", "Low", "High"]),
                 "SC_category_H3N2": (h3n2, ["None", "Low", "High"]),
                 "SC_category_IBV_Yam": (yam, ["None", "Low", "High"]),
                 "SC_category_IBV_Vic": (vic, ["None", "Low", "High"]),
                 "Composite_SC_category": (comp, ["None", "Low", "High"]),
                 "Baseline_category_Num_SeroPos_strains": (base, ["Low", "High"])}  # mapping from columns to correct ordering
else:
  ordering_dict = {"BMI_category": (bmi_cat, ["Lean", "Normal", "Overweight", "Obese"]),
                 "Comorbidities": (comorbidities, ["No", "Yes"]),
                 "Vaccine_dose": (dose, ["Standard", "High"]),
                 "PreVacc_status": (pre_vac, ["Naive", "Prevaccinated"]),
                 "SC_category_H1N1": (h1n1, ["None", "Low", "High"]),
                 "SC_category_H3N2": (h3n2, ["None", "Low", "High"]),
                 "SC_category_IBV_Yam": (yam, ["None", "Low", "High"]),
                 "SC_category_IBV_Vic": (vic, ["None", "Low", "High"]),
                 "Composite_SC_category": (comp, ["None", "Low", "High"]),
                 "Baseline_category_Num_SeroPos_strains": (base, ["Low", "High"])}  
                 
lencoder = CustomLabelEncoder()
for key, value in ordering_dict.items():
  lencoder.fit(value[0], value[1])
  df.loc[:, key] = lencoder.transform(value[0])

df






,Cohort,ID,Age,BMI,BMI_category,Comorbidities,PreVacc_status,Vaccine_dose,D0_Titer_H1N1,D28_Titer_H1N1,...,SC_category_IBV_Yam,SC_category_IBV_Vic,Composite_SC_category,Serostatus_H1N1,Serostatus_H3N2,Serostatus_IBV_Yam,Serostatus_IBV_Vic,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline
0,UGA1,A10,-0.622445,1.039092,3,0,1,0,0.637952,160,...,0,0,0,1,1,1,1,4,1,35.00
1,UGA1,A100,0.429208,0.986051,3,0,1,0,-0.448919,40,...,0,2,1,0,0,1,0,1,0,18.00
2,UGA1,A101,-0.383433,-0.321247,1,1,0,0,-0.448919,640,...,2,2,2,0,0,1,1,2,0,21.00
3,UGA1,A102,0.429208,-0.319687,2,0,0,0,0.016883,160,...,1,2,2,1,1,1,0,3,1,25.00
4,UGA1,A103,-0.861457,-0.772093,1,0,0,0,-0.293651,640,...,2,2,2,1,1,1,1,4,1,21.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,UGA5,C90,-1.148271,-1.135578,1,0,1,0,0.016883,40,...,1,0,0,1,0,1,0,2,0,20.29
1364,UGA5,C93,-1.148271,-1.341501,0,0,1,0,-0.448919,160,...,2,1,1,0,1,1,0,2,0,20.29
1365,UGA5,C94,-1.100469,-0.208925,2,0,1,0,-0.293651,80,...,2,2,2,1,0,1,1,3,1,20.29
1366,UGA5,C97,-1.004864,-0.586451,1,0,1,0,0.016883,160,...,1,2,1,1,1,1,1,4,1,26.29


* Combine two parts of the data

In [87]:
new = df.join(x)
new

,Cohort,ID,Age,BMI,BMI_category,Comorbidities,PreVacc_status,Vaccine_dose,D0_Titer_H1N1,D28_Titer_H1N1,...,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline,Gender_Male,Race_White,Month_vaccinated_Feb.,Month_vaccinated_Jan.,Month_vaccinated_Nov.,Month_vaccinated_Oct.,Month_vaccinated_Sep.
0,UGA1,A10,-0.622445,1.039092,3,0,1,0,0.637952,160,...,4,1,35.00,1,1,0,1,0,0,0
1,UGA1,A100,0.429208,0.986051,3,0,1,0,-0.448919,40,...,1,0,18.00,0,1,0,1,0,0,0
2,UGA1,A101,-0.383433,-0.321247,1,1,0,0,-0.448919,640,...,2,0,21.00,1,0,0,1,0,0,0
3,UGA1,A102,0.429208,-0.319687,2,0,0,0,0.016883,160,...,3,1,25.00,1,1,1,0,0,0,0
4,UGA1,A103,-0.861457,-0.772093,1,0,0,0,-0.293651,640,...,4,1,21.00,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,UGA5,C90,-1.148271,-1.135578,1,0,1,0,0.016883,40,...,2,0,20.29,0,1,0,0,0,1,0
1364,UGA5,C93,-1.148271,-1.341501,0,0,1,0,-0.448919,160,...,2,0,20.29,1,1,0,0,0,1,0
1365,UGA5,C94,-1.100469,-0.208925,2,0,1,0,-0.293651,80,...,3,1,20.29,1,1,0,0,1,0,0
1366,UGA5,C97,-1.004864,-0.586451,1,0,1,0,0.016883,160,...,4,1,26.29,1,1,0,0,0,1,0


* Derive two version of the data for future compatibility of omics data

In [88]:
new_UGA4 = new[new.iloc[:,0] == "UGA4"]  # get two versions of the data: UGA4 and full version; drop the ID columns
new_UGA4_strip = new_UGA4.iloc[:,2:]
new_strip = new.iloc[:,2:]

In [89]:
new_strip

,Age,BMI,BMI_category,Comorbidities,PreVacc_status,Vaccine_dose,D0_Titer_H1N1,D28_Titer_H1N1,D0_Titer_H3N2,D28_Titer_H3N2,...,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline,Gender_Male,Race_White,Month_vaccinated_Feb.,Month_vaccinated_Jan.,Month_vaccinated_Nov.,Month_vaccinated_Oct.,Month_vaccinated_Sep.
0,-0.622445,1.039092,3,0,1,0,0.637952,160,3.591594,640,...,4,1,35.00,1,1,0,1,0,0,0
1,0.429208,0.986051,3,0,1,0,-0.448919,40,-0.447529,40,...,1,0,18.00,0,1,0,1,0,0,0
2,-0.383433,-0.321247,1,1,0,0,-0.448919,640,-0.447529,320,...,2,0,21.00,1,0,0,1,0,0,0
3,0.429208,-0.319687,2,0,0,0,0.016883,160,-0.317235,80,...,3,1,25.00,1,1,1,0,0,0,0
4,-0.861457,-0.772093,1,0,0,0,-0.293651,640,-0.317235,320,...,4,1,21.00,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,-1.148271,-1.135578,1,0,1,0,0.016883,40,-0.447529,40,...,2,0,20.29,0,1,0,0,0,1,0
1364,-1.148271,-1.341501,0,0,1,0,-0.448919,160,-0.317235,80,...,2,0,20.29,1,1,0,0,0,1,0
1365,-1.100469,-0.208925,2,0,1,0,-0.293651,80,-0.512677,320,...,3,1,20.29,1,1,0,0,1,0,0
1366,-1.004864,-0.586451,1,0,1,0,0.016883,160,0.464531,320,...,4,1,26.29,1,1,0,0,0,1,0


In [90]:
new_UGA4_strip

,Age,BMI,BMI_category,Comorbidities,PreVacc_status,Vaccine_dose,D0_Titer_H1N1,D28_Titer_H1N1,D0_Titer_H3N2,D28_Titer_H3N2,...,Num_SeroPos_strains,Baseline_category_Num_SeroPos_strains,Composite_baseline,Gender_Male,Race_White,Month_vaccinated_Feb.,Month_vaccinated_Jan.,Month_vaccinated_Nov.,Month_vaccinated_Oct.,Month_vaccinated_Sep.
597,-0.335630,1.728621,3,0,1,0,-0.293651,80,-0.545250,160,...,3,1,20.0,0,1,0,0,0,1,0
598,0.524813,0.986051,3,0,1,0,-0.565369,20,-0.447529,160,...,0,0,14.0,0,1,0,0,0,1,0
599,-0.479038,1.853423,3,0,1,0,0.637952,160,-0.512677,20,...,3,1,26.0,0,1,0,1,0,0,0
600,-0.622445,-0.903135,1,0,1,0,-0.293651,80,-0.317235,320,...,4,1,23.0,0,1,0,0,0,1,0
601,1.815479,-0.165245,2,1,1,1,-0.565369,10,-0.545250,10,...,0,0,12.0,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,-1.148271,-0.825134,1,0,1,0,-0.448919,80,-0.056647,640,...,3,1,24.0,0,1,0,0,0,1,0
1033,-1.196074,-1.224499,1,0,1,0,1.880089,320,-0.545250,20,...,3,1,21.0,0,1,0,0,0,1,0
1034,-1.196074,-1.339941,0,0,1,0,0.637952,160,-0.317235,40,...,3,1,20.0,1,1,0,0,0,1,0
1035,-1.052667,-0.594251,1,0,1,0,1.880089,320,-0.056647,160,...,4,1,29.0,1,1,0,0,1,0,0


* Normalization

# Input/Output selector

* Select which columns to be the input. Refer to the columns as their original names.

* Choose output type $\texttt{tuple(type: str, antigen: str)}$

For type:
 * Seroconversion: regression
 * D28_Titer: regression
 * SC_category: classification
 * Serostatus*: classification 

For antigen:
 * H1N1
 * H3N2
 * IBV_Yam
 * IBV_Vic
 * Composite

\* when antigen is composite, output is $\texttt{high}$ or $\texttt{low}$ for Seropositive strains.



In [91]:
def io_selector(data: DataFrame, 
                input_columns: Union[list, str] = None, # 'all' for all input columns. Used for feature selection
                output_column: str = None, 
                month_one_hot: bool = True) -> Tuple[pd.DataFrame, ...]:
  
  
  complete_input_columns = ["Age", "BMI", "BMI_category", "Gender_Male", "Race_White", 
                              "Comorbidities", "PreVacc_status", "PreVacc_status_year", 
                             "Vaccine_dose", "D0_Titer_H1N1", 
                           "D0_Titer_H3N2", "D0_Titer_IBV_Yam", "D0_Titer_IBV_Vic"]

  default_input_columns = ["Age", "BMI", "Gender_Male", "Race_White", "Comorbidities", 
                           "PreVacc_status", "Vaccine_dose",
                           "D0_Titer_H1N1", "D0_Titer_H3N2", "D0_Titer_IBV_Yam", 
                           "D0_Titer_IBV_Vic"]
  name_mapping = {"Gender": ["Gender_Male"], "Race": ["Race_White"]}
  if month_one_hot:
    default_input_columns.extend(["Month_vaccinated_Jan.", "Month_vaccinated_Feb.",
                           "Month_vaccinated_Sep.", "Month_vaccinated_Oct.",
                           "Month_vaccinated_Nov."])
    complete_input_columns.extend(["Month_vaccinated_Jan.", "Month_vaccinated_Feb.",
                           "Month_vaccinated_Sep.", "Month_vaccinated_Oct.",
                           "Month_vaccinated_Nov."])
    name_mapping["Month_vaccinated"] = ["Month_vaccinated_Jan.",
                                       "Month_vaccinated_Feb.",
                                       "Month_vaccinated_Sep.", 
                                       "Month_vaccinated_Oct.", 
                                       "Month_vaccinated_Nov."]
  else:
    default_input_columns.append("Month_vaccinated")
    complete_input_columns.append("Month_vaccinated")
    
                           
  default_output = ["Composite_seroconversion"]

  
  if input_columns is None: 
    input_columns = default_input_columns
  elif input_columns == "all":
    input_columns = complete_input_columns
  else:
    temp = []
    for item in input_columns:
      if item in name_mapping:
        temp.extend(name_mapping[item])
      else:
        temp.append(item)
    input_columns = temp
  if output_column is None:
    output_column = default_output
  x = data[input_columns]
  y = data[output_column]
  return (x, y)


Test to see whether it works

In [92]:
io_selector(new_strip)[1].columns[0]

'Composite_seroconversion'

# Setting up models


Wrappers for dataset and training. This will make future validation very convenient.

In [93]:
def dataset_wrapper(data: DataFrame, 
                input_columns: list = None, 
                output_column: str = None,
                train: float = 0.8,  # train, test split ratio, must add up to 1
                test: float = 0.2,
                random_state: int = 10,  # random seed
                save: bool = False,  # whether to save data in cwd
                name_extension: str = "",  # name extension for file save
                ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:  # returns X_train, y_train, X_dev, y_dev, X_test, y_test
  random.seed(random_state)
  assert train + test == 1.0
  X, y = io_selector(data, input_columns, output_column)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test, random_state=random_state)
  if save:
    X_train.to_csv("X_train_" + name_extension + ".csv", index=False)
    y_train.to_csv("y_train_" + name_extension + ".csv", index=False)
    X_test.to_csv("X_test_" + name_extension + ".csv", index=False)
    y_test.to_csv("y_test_" + name_extension + ".csv", index=False)
  X_train = X_train.to_numpy()
  y_train = y_train.to_numpy()
  X_test = X_test.to_numpy()
  y_test = y_test.to_numpy()
  return X_train, y_train, X_test, y_test

In [94]:
import sys
X_train, y_train = dataset_wrapper(new_strip)[:2]
np.set_printoptions(threshold=sys.maxsize)
X_train

array([[ 8.11627791e-01,  8.64369421e-01,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -5.65368904e-01,  4.64530595e-01,
        -5.37311690e-01, -6.82689305e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [-5.26839979e-01, -3.43087293e-01,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         0.00000000e+00, -4.48918516e-01, -5.66465880e-02,
         2.44398819e-01, -6.23610423e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [-1.14827144e+00, -2.83806472e-01,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.88008925e+00,  4.64530595e-01,
        -3.80969588e-01, -2.69137130e-01,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 2.05449072e+00,  8.05088600e

Model name docs:
* LR: Linear Regressor
* SGDC/SGDR: Stochastic Gradient Descent Classifier/Regressor
* LOG: Logistic Regression Classifier
* RFC/RFR: Random Forest Classifier/Regressor
* SVC/SVR: Support Vector Machine Classifier/Regressor
* KNNC/KNNR: K Nearest Neighbors Classifier/Regressor

In [95]:
from sklearn.linear_model import LinearRegression, SGDRegressor, SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

def train_wrapper(X_train: np.ndarray,  # dataset
                  y_train: np.ndarray,
                # Universal parameters
                method: str = None,  # model name
                random_state: int = 10,  # random seed
                model_path: str = None,  # path to save the model, no saving if None
                n_jobs: int = -1,  # number of CPUs to use for some models. -1 means all processors
                max_iter: int = 10000,  # max number of iterations for some models, set to -1 for SV to impose no limits 
                class_weight: Union[dict, str] = None,  # weight of each class or balanced
                penalty: str = "l2",  # logistic has the option of no penalty "None"
                # Model specific parameters:
                # SGD Regressor/Classifier
                loss: str = "squared_error",
                alpha: float = 0.01,
                l1_ratio: float = 0.15,
                SGD_epsilon: float = 0.1,
                learning_rate: str = "invscaling",
                eta0: float = 1e-2,
                power_t: float = 0.25,
                average: Union[bool, int] = False,
                # KNN Regressor/Classifier
                n_neighbors: int = 5,
                weights: str = "uniform",
                algorithm: str = "brute",
                leaf_size: int = 30,
                p: int = 2,
                metric: Union[str, callable] = "minkowski",
                metric_params: dict = None,
                # Logistic Regression
                dual: bool = False,
                C: float = 1.0,
                intercept_scaling: float = 1,
                solver: str = "lbfgs",
                multi_class: str = "auto",
                # Random Forest Classifier/Regressor
                n_estimates: int = 100,
                criterion: str = "squared_error",
                max_depth: int = None,
                min_sample_split: Union[int, float] = 2,
                min_samples_leaf: Union[int, float] = 1,
                min_weight_fraction_leaf: float = 0.0,
                max_features: Union[str, int, float] = "auto",
                max_leaf_nodes: int = None,
                min_impurity_decrease: float = 0.0,
                bootstrap: bool = True,
                oob_score: bool = True,
                ccp_alpha: float = 0.0,
                max_samples: Union[int,float] = None,
                # SVC/SVR
                kernel: str = "rbf",
                degree: int = 3,
                gamma: str = "scale",
                coef0: float = 0.0,
                shrinking: bool = True,
                probability: bool = False,
                decision_fuction_shape: str = "ovr",
                break_ties: bool = False,
                SVC_epsilon: float = 0.1,
                ):  # returns a trained model
  random.seed(random_state)
  model = LinearRegression()
  if method == "LR":
    model = LinearRegression()
  elif method == "SGDR":
    model = SGDRegressor(loss=loss, penalty=penalty, alpha=alpha, l1_ratio=l1_ratio, max_iter=max_iter, verbose=0, epsilon=SGD_epsilon, random_state=random_state, learning_rate=learning_rate, eta0=eta0, power_t=power_t, average=average)
  elif method == "KNNR":
    model = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, algorithm=algorithm, leaf_size=leaf_size, p=p, metric=metric)
  elif method == "SGDC":
    model = SGDClassifier(loss=loss, penalty=penalty, alpha=alpha, l1_ratio=l1_ratio, max_iter=max_iter, verbose=0, epsilon=SGD_epsilon, n_jobs=n_jobs, random_state=random_state, learning_rate=learning_rate, eta0=eta0, power_t=power_t, class_weight=class_weight, average=average)
  elif method == "KNNC":
    model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, algorithm=algorithm, leaf_size=leaf_size, p=p, metric=metric)
  elif method == "LOGR":
    model = LogisticRegression(penalty=penalty, dual=dual, C=C, intercept_scaling=intercept_scaling, class_weight=class_weight, random_state=random_state, solver=solver, max_iter=max_iter, multi_class=multi_class, verbose=1, n_jobs=n_jobs, l1_ratio=l1_ratio)
  elif model == "RFC":
    model = RandomForestClassifier(n_estimators=n_estimates, criterion=criterion, max_depth=max_depth, min_samples_split=min_sample_split, min_samples_leaf=min_samples_leaf, min_weight_fraction_leaf=min_weight_fraction_leaf, max_features=max_features, max_leaf_nodes=max_leaf_nodes, min_impurity_decrease=min_impurity_decrease, bootstrap=bootstrap, oob_score=oob_score, n_jobs=n_jobs, random_state=random_state, verbose=1, class_weight=class_weight, ccp_alpha=ccp_alpha, max_samples=max_samples)
  elif model == "RFR":
    model = RandomForestRegressor(n_estimators=n_estimates, criterion=criterion, max_depth=max_depth, min_samples_split=min_sample_split, min_samples_leaf=min_samples_leaf, min_weight_fraction_leaf=min_weight_fraction_leaf, max_features=max_features, max_leaf_nodes=max_leaf_nodes, min_impurity_decrease=min_impurity_decrease, bootstrap=bootstrap, oob_score=oob_score, n_jobs=n_jobs, random_state=random_state, verbose=1, ccp_alpha=ccp_alpha, max_samples=max_samples)
  elif method == "SVC":
    model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma, coef0=coef0, shrinking=shrinking, probability=probability, class_weight=class_weight, verbose=1, max_iter=max_iter, decision_function_shape=decision_fuction_shape, break_ties=break_ties, random_state=random_state)
  elif method == "SVR":
    model = SVR(kernel=kernel, degree=degree, gamma=gamma, coef0=coef0, C=C, shrinking=shrinking, epsilon=SVC_epsilon, verbose=1, max_iter=max_iter)
  model.fit(X_train,y_train)
  if model_path:
    with open(method + uuid.uuid1()[:6] + ".pkl", "wb") as f:
      pickle.dump(model, f)
  return model


In [96]:
model = train_wrapper(X_train, y_train)
model.coef_

array([[-5.62830685e-01,  3.46986838e-01, -3.65270503e-01,
        -1.35894050e-03, -4.96576330e-02, -6.42955431e+00,
         9.23770822e-01, -7.33913600e-01, -3.48974772e-01,
        -6.23181396e-01, -8.09554415e-01, -4.12685196e-01,
         7.55873879e-01, -1.13529789e+00, -1.34112804e+00,
        -7.94781311e-01]])

# Regression task: composite seroconversion
> Performance of different models based on $R^2$

In [97]:
X_train, y_train, X_test, y_test = dataset_wrapper(new_strip)
model_name_list = ["LR", "KNNR", "SGDR", "RFR", "SVR"]
for model_name in model_name_list:
  model = train_wrapper(X_train, y_train, method=model_name)
  print(model_name,"has R squared score ", model.score(X_test, y_test))


LR has R squared score  0.36149253099485623
KNNR has R squared score  0.4121322232837831
SGDR has R squared score  0.3643140555893334
RFR has R squared score  0.36149253099485623
[LibSVM]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR has R squared score  0.43116204000415304


Comparison using the same metric as Wu

# Classification task: $\texttt{Composite_SC_category}$
>Performance of different models based on accuracy

In [98]:
X_train, y_train, X_test, y_test = dataset_wrapper(new_strip, output_column= "Composite_SC_category")
model_name_list = ["LOGR", "KNNC", "SGDC", "RFC", "SVC"]
for model_name in model_name_list:
  model = train_wrapper(X_train, y_train, method=model_name)
  print(model_name,"has accuracy ", model.score(X_test, y_test))


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1479: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.8s finished


LOGR has accuracy  0.5656934306569343
KNNC has accuracy  0.5255474452554745
SGDC has accuracy  0.5656934306569343
RFC has accuracy  0.28581128865916394
[LibSVM]SVC has accuracy  0.5766423357664233


More experiments (cross validations, metrics) coming soon.

# $\texttt{Month_vaccinated}$ encoding methods
* Month one hot encoding:

>>LR has R squared score  0.36149253099485634 \
KNNR has R squared score  0.4121322232837831\
SGDR has R squared score  -0.7037487125710182\
RFR has R squared score  0.36149253099485634\
[LibSVM]SVR has R squared score  0.43116204000415304\
LOGR has accuracy  0.5656934306569343\
KNNC has accuracy  0.5255474452554745\
SGDC has accuracy  0.4708029197080292\
RFC has accuracy  0.28581128865916383\
[LibSVM]SVC has accuracy  0.5766423357664233

* Month binary encoding(Sep to Nov as 0 and Dec to Feb as 1):
>LR has R squared score  0.3541087354194813\
KNNR has R squared score  0.36461163211116476\
SGDR has R squared score  -0.6949622914919593\
RFR has R squared score  0.3541087354194813\
[LibSVM]SVR has R squared score  0.424852045067729\
LOGR has accuracy  0.5985401459854015\
KNNC has accuracy  0.5437956204379562\
SGDC has accuracy  0.4781021897810219\
RFC has accuracy  0.27657348805709536\
[LibSVM]SVC has accuracy  0.5656934306569343

* Month label encoding:
>LR has R squared score  0.35892487763453085\
KNNR has R squared score  0.40670900113569797\
SGDR has R squared score  -0.17643970183725943\
RFR has R squared score  0.35892487763453085\
[LibSVM]SVR has R squared score  0.4284985926239723\
LOGR has accuracy  0.5985401459854015\
KNNC has accuracy  0.5145985401459854\
SGDC has accuracy  0.46715328467153283\
RFC has accuracy  0.278981729332912\
[LibSVM]SVC has accuracy  0.5766423357664233

Turns out that there's no severe semantic penalty using label encoding, binary encoding did not make much sense, and one hot encoding hurts dimension-sensitive models. So, I'm preserving methods to generate both one-hot and label encoding datasets.

\*By the time this issue was investigated, SGDR had been broken but is now fixed.

